In [42]:
from pynytimes import NYTAPI
import pyinputplus as pyip
import json
import pprint
import requests
from datetime import datetime, timedelta
import time 
import collections 
import dateutil.parser
import webbrowser
"""import anvil.server

anvil_key = '3BZLXRGRMCFTO4WZW26NAQPM-2K4TP7HXOBVG6CZE'
anvil.server.connect(anvil_key)"""

#access to API
key = 'ESClZEYkPOiIymPbcHCIe4pO7su9Ua2m'
nyt = NYTAPI(key)

#initialize variables
start_date = ""
end_date = ""
book_list_name = ""
isbn13_list = []
isbn13_count = {}
recommended_books = []
current_date = ""
number_recs = 0

#simple function to validate user date inputs 
def is_valid_regex(num):
    while type(num) == int:
        print('Incorrect input. Please format as YYYY-MM-DD.')
        num = pyip.inputNum(allowRegexes=[r'^\d{4}\-(0?[1-9]|1[012])\-(0?[1-9]|[12][0-9]|3[01])$'])
    return num

def is_valid_date(x):
    try:
        dt = datetime.strptime(x,"%Y-%m-%d")
        valid_datetime = dateutil.parser.parse(x)
        return True
    except ValueError: 
        return False

#get start date, make sure it's a valid date
print('What is your start date? Please format as YYYY-MM-DD and insert a date that is after 2011-02-06.')
start_date = is_valid_regex(pyip.inputNum(allowRegexes=[r'^\d{4}\-(0?[1-9]|1[012])\-(0?[1-9]|[12][0-9]|3[01])$']))

while is_valid_date(start_date) == False:
    print('That is not a valid date. Please try again.')
    start_date = is_valid_regex(pyip.inputNum(allowRegexes=[r'^\d{4}\-(0?[1-9]|1[012])\-(0?[1-9]|[12][0-9]|3[01])$']))

#make sure date is not too far back
while datetime.strptime(start_date, "%Y-%m-%d") <= datetime.strptime('2011-02-06',"%Y-%m-%d"):
    print ("Please enter a date after 2011-02-06. Try again.")
    start_date = is_valid_regex(pyip.inputNum(allowRegexes=[r'^\d{4}\-(0?[1-9]|1[012])\-(0?[1-9]|[12][0-9]|3[01])$']))

    if is_valid_date(start_date) == False:
        print('That is not a valid date. Please try again.')
        start_date = is_valid_regex(pyip.inputNum(allowRegexes=[r'^\d{4}\-(0?[1-9]|1[012])\-(0?[1-9]|[12][0-9]|3[01])$']))
    
#get end date and make sure they do not enter a number b/c that triggers an error; check it is not in the future
print('What is your end date? Please format as YYYY-MM-DD.')
end_date = is_valid_regex(pyip.inputNum(allowRegexes=[r'^\d{4}\-(0?[1-9]|1[012])\-(0?[1-9]|[12][0-9]|3[01])$']))

while is_valid_date(end_date) == False:
    print('That is not a valid date. Please try again.')
    end_date = is_valid_regex(pyip.inputNum(allowRegexes=[r'^\d{4}\-(0?[1-9]|1[012])\-(0?[1-9]|[12][0-9]|3[01])$']))

while datetime.strptime(end_date, "%Y-%m-%d") > datetime.today():
    print ("Please do not enter a date in the future. Try again.")
    end_date = is_valid_regex(pyip.inputNum(allowRegexes=[r'^\d{4}\-(0?[1-9]|1[012])\-(0?[1-9]|[12][0-9]|3[01])$']))
    
    while is_valid_date(end_date) == False:
        print('That is not a valid date. Please try again.')
        end_date = is_valid_regex(pyip.inputNum(allowRegexes=[r'^\d{4}\-(0?[1-9]|1[012])\-(0?[1-9]|[12][0-9]|3[01])$']))

#make sure start date is before end date
while datetime.strptime(start_date,"%Y-%m-%d") > datetime.strptime(end_date,"%Y-%m-%d"):
    print('Please enter an end date that is after the start date. Please format as YYYY-MM-DD.')
    end_date = is_valid_regex(pyip.inputNum(allowRegexes=[r'^\d{4}\-(0?[1-9]|1[012])\-(0?[1-9]|[12][0-9]|3[01])$']))
    
    while is_valid_date(end_date) == False:
        print('That is not a valid date. Please try again.')
        end_date = is_valid_regex(pyip.inputNum(allowRegexes=[r'^\d{4}\-(0?[1-9]|1[012])\-(0?[1-9]|[12][0-9]|3[01])$']))

#check what kind of books they would like 
print('What types of books would you like? Please indicate fiction or non-fiction')
book_input = input()

while True:
    if book_input.lower() == "fiction":
        book_list_name = "fiction"
        break
    elif book_input.lower() == "non-fiction" or book_input.lower()=="nonfiction":
        book_list_name = "nonfiction"
        break
    else:
        print('Please enter either fiction or non-fiction.')
        book_input = input()

#check how many books they want returned in rec list 
number_recs = pyip.inputNum('How many recommendations would you like? ', min=0)

#get dates between start and end date into list 
current_date = datetime.strptime(start_date, "%Y-%m-%d")
end_date = datetime.strptime(end_date, "%Y-%m-%d")
while current_date <= end_date:
    isbn13_list.append(str(current_date.strftime("%Y-%m-%d")))
    current_date += timedelta(days=7)

Connecting to wss://anvil.works/uplink
Anvil websocket open
Authenticated OK
What is your start date? Please format as YYYY-MM-DD and insert a date that is after 2011-02-06.


KeyboardInterrupt: 

In [39]:
#get isbn13s for all best sellers and put into a dictionary; count frequency of appearance week to week 
isbn13_count = {}

for i in isbn13_list:
    best_sellers = nyt.best_sellers_list(name = 'combined-print-and-e-book-' + book_list_name, date = datetime.strptime(i, '%Y-%m-%d' ))
    y = 0
    while y < len(best_sellers):
        current = str(best_sellers[y]['title']) + " " + str(best_sellers[y]['buy_links'])
        if current not in isbn13_count.keys():
            isbn13_count[current]=1
        else:
            isbn13_count[current]+=1
        #give API time to run 
        #time.sleep(2)
        y += 1

#sort by number of times the isbn13 appears in the dictionary 
ranked_isbn13 = collections.OrderedDict(isbn13_count)
ranked_isbn13 = sorted(ranked_isbn13.items(), key = lambda x: x[1], reverse = True)

In [40]:
print("Your book recommendations are: \n") 
count = 0 

try:
    while count < number_recs:
        #title is going to be the first thing in the list, full string, split by "[", first element
        title = ranked_isbn13[count][0].split('[')[0].title()
        if ranked_isbn13[count][1] == 1:
            print(title[:len(title)-1] + ", appearing " + str(ranked_isbn13[count][1]) + " time")
        else: 
            print(title[:len(title)-1] + ", appearing " + str(ranked_isbn13[count][1]) + " times")
        count += 1
    
    print('-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*\n')
    #ask people if they want the next book IF that book appeared as many times as the last book on their recommended list 
    while ranked_isbn13[count-1][1] == ranked_isbn13[count][1]:
        print("The next book appeared the same number of times as the last book on your recommendations list. Would you like to see it?")
        response = input()

        #validate their input 
        while response.lower() != "yes" and response.lower() != "no":
            print('Please respond either yes or no')
            response = input()

        #if they say they wanted another book
        if response.lower() == "yes":
            title = ranked_isbn13[count][0].split('[')[0].title()
            if ranked_isbn13[count][1] == 1:
                print(title[:len(title)-1] + ", appearing " + str(ranked_isbn13[count][1]) + " time")
            else: 
                print(title[:len(title)-1] + ", appearing " + str(ranked_isbn13[count][1]) + " times")
            count += 1
        elif response.lower() == 'no':
            break
    
except IndexError:
    print("You have reached the end of the list!\n")

print('-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*\n')
print("Your final book list is:\n")

recommended_books = []
x = 0 
while x < count:
    #this is so brute forced...have to think of better way to do this next time
    title = ranked_isbn13[x][0].split('[')[0].title()
    links = ranked_isbn13[0][0].split('[')[1].split('] ')[0]
    links = ranked_isbn13[0][0].split('[')[1].split('] ')[0]
    current_entry = [title, links]
    recommended_books.append(current_entry)
    if ranked_isbn13[x][1] == 1:
        print(title[:len(title)-1] + ", appearing " + str(ranked_isbn13[x][1]) + " time")
    else: 
        print(title[:len(title)-1] + ", appearing " + str(ranked_isbn13[x][1]) + " times")
    x+=1

Your book recommendations are: 

Where The Crawdads Sing, appearing 6 times
Little Fires Everywhere, appearing 6 times
The Silent Patient, appearing 5 times
American Dirt, appearing 4 times
The Mirror & The Light, appearing 4 times
Blindside, appearing 3 times
The Dutch House, appearing 3 times
The Tattooist Of Auschwitz, appearing 3 times
The Boy From The Woods, appearing 3 times
The Glass Hotel, appearing 3 times
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

Your final book list is:

Where The Crawdads Sing, appearing 6 times
Little Fires Everywhere, appearing 6 times
The Silent Patient, appearing 5 times
American Dirt, appearing 4 times
The Mirror & The Light, appearing 4 times
Blindside, appearing 3 times
The Dutch House, appearing 3 times
The Tattooist Of Auschwitz, appearing 3 times
The Boy From The Woods, appearing 3 times
The Glass Hotel, appearing 3 times


In [41]:
print("Would you like to open links to purchase these books?")

response = str(input())

#validate their input 
while response.lower() != "yes" and response.lower() != "no":
    print('Please respond either yes or no')
    response = input()

if response.lower() == 'no':
    print('Sounds good. Exiting now!')
    
elif response.lower() == 'yes':
    print('How many links would you like to be opened? Please specify a number.')
    print_books = pyip.inputNum()
    
    while int(print_books) > len(recommended_books):
        print('You are only able to receive as many links are there are books on your recommended list. Please try again')
        print_books = pyip.inputNum()

    x = 0

    while x < print_books:
        links = ranked_isbn13[x][0].split('[')[1].split(']')[0]
        separated = links.split(', ')
        indie = separated[11]
        indie_final = indie[8:len(indie)-2]
        webbrowser.open(indie_final)
        x+=1


Would you like to open links to purchase these books?


Please respond either yes or no


 yes


How many links would you like to be opened? Please specify a number.


 2
